In [18]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import os
sns.set()
# import the KMeans clustering model from scikit-learn
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
df = pd.read_csv('https://mda-project-poland.s3.eu-west-3.amazonaws.com/ultimate+aquastat+(flat).csv')  # will be used for Time series analysis
df_GDP = pd.read_csv('https://mda-project-poland.s3.eu-west-3.amazonaws.com/GDP.csv')
df_2018 = df.loc[df['Year'] == 2018]  # will be used for classification

In [20]:

table_2018 = pd.pivot_table(data=df_2018,values='Value',index='Area',columns='Variable Name')

# drop undernourishment and add GDP  
table_2018 = table_2018.drop('Prevalence of undernourishment (3-year average)', axis=1)  # TODO download correct data from AQUASTAT instead of dropping here
df_GDP_2018 = df_GDP.loc[df_GDP['Year'] == 2018] 
table_GDP_2018 = pd.pivot_table(data=df_GDP_2018,values='Value',index='Area',columns='Variable Name')

table_2018 = table_2018.merge(table_GDP_2018, how='inner', on='Area')

table_2018.head() 

Variable Name,Agricultural water withdrawal as % of total renewable water resources,"Agriculture, value added (% GDP)",Average Temperature,Environmental Flow Requirements,GDP per capita,"Industry, value added to GDP",Long-term average annual precipitation in volume,MDG 7.5. Freshwater withdrawal as % of total renewable water resources,National Rainfall Index (NRI),Population density,SDG 6.4.1. Industrial Water Use Efficiency,SDG 6.4.1. Irrigated Agriculture Water Use Efficiency,SDG 6.4.1. Services Water Use Efficiency,SDG 6.4.1. Water Use Efficiency,SDG 6.4.2. Water Stress,"Services, value added to GDP",Total population with access to safe drinking-water (JMP),Total renewable water resources per capita,Total water withdrawal per capita,Gross Domestic Product (GDP)
Area,,,,,,,,,,,,,,,,,,,,
Afghanistan,30.613807,22.440899,14.55,28.29,563.983834,4.800000e+09,213.48522,31.045461,185.0,56.937048,31.220059,0.102132,57.683901,0.935917,54.757019,1.060000e+10,55.3,1757.509385,548.072294,1.798697e+10
Argentina,3.187483,6.097129,14.99,515.80,11601.890080,1.070000e+11,1643.21640,4.301333,1062.0,15.954953,32.596217,0.152694,62.519941,13.244872,10.456664,3.010000e+11,99.1,19752.418500,851.646091,5.198717e+11
Australia,2.444073,2.121597,22.58,243.30,58689.490190,3.020000e+11,4133.81148,2.354494,717.1,3.216309,91.638244,0.418891,405.626370,70.079929,4.657865,1.010000e+12,100.0,19760.502710,697.308780,1.458953e+12
Azerbaijan,32.406633,5.212381,14.16,12.03,4729.900727,2.400000e+10,38.71020,35.088681,459.7,114.890727,50.317916,0.159418,46.604281,3.870952,53.729300,1.680000e+10,87.0,3485.086793,1222.870974,4.711248e+10
Bolivia (Plurinational State of),0.334495,11.481924,20.80,396.60,3592.084429,9.720000e+09,1258.97268,0.363763,1503.0,10.334379,281.746033,0.248811,140.699591,13.711080,1.177001,2.060000e+10,90.0,50558.691150,183.913845,4.028765e+10


In [21]:
table=pd.pivot_table(data=df,values='Value',index=['Area','Year'],columns='Variable Name')

# drop undernourishment and add GDP  
table = table.drop('Prevalence of undernourishment (3-year average)', axis=1)  # TODO download correct data from AQUASTAT instead of dropping here
table_GDP = pd.pivot_table(data=df_GDP,values='Value',index=['Area','Year'],columns='Variable Name')
table
table = table.merge(table_GDP, how='inner', on=['Area','Year'])
table.head(21)

Variable Name       Agricultural water withdrawal as % of total renewable water resources  \
Area        Year                                                                            
Afghanistan 1992.0                                          35.498101                       
            1997.0                                          31.427856                       
            2002.0                                          30.613807                       
            2007.0                                          30.613807                       
            2012.0                                          30.613807                       
            2017.0                                          30.613807                       
            2018.0                                          30.613807                       
Argentina   1992.0                                                NaN                       
            1997.0                                           2.455948                       
            2002.0                                           2.588954                       
            2007.0                                           2.921470                       
            2012.0                                           3.187483                       
            2017.0                                           3.187483                       
            2018.0                                           3.187483                       
Australia   1992.0                                                NaN                       
            1997.0                                           3.131301                       
            2002.0                                           2.913110                       
            2007.0                                           1.987195                       
            2012.0                                           1.948577                       
            2017.0                                           2.250000                       
            2018.0                                           2.444073                       

Variable Name       Agriculture, value added (% GDP)  Average Temperature  \
Area        Year                                                            
Afghanistan 1992.0                         54.207106                12.61   
            1997.0                         62.416768                12.86   
            2002.0                         45.134344                13.91   
            2007.0                         35.265942                13.80   
            2012.0                         28.556142                13.25   
            2017.0                         25.574670                13.12   
            2018.0                         22.440899                14.55   
Argentina   1992.0                          5.193747                14.37   
            1997.0                          4.527197                15.14   
            2002.0                          8.848660                14.60   
            2007.0                          7.452354                14.20   
            2012.0                          5.781744                15.20   
            2017.0                          5.478382                15.30   
            2018.0                          6.097129                14.99   
Australia   1992.0                          3.263718                21.75   
            1997.0                          3.038954                21.82   
            2002.0                          2.888534                22.25   
            2007.0                          2.338630                22.08   
            2012.0                          2.278906                21.83   
            2017.0                          2.458346                22.57   
            2018.0                          2.121597                22.58   

Variable Name       Environmental Flow Requirements  GDP per capita  \
Area        Year 

## Data cleaning

The only missing value in the 2018 dataset is the National Rainfall Index (NRI) in North Korea.

In [22]:
table_2018_preprocessed = table_2018.fillna(table_2018.mean())
# TODO maybe replace with some manually calculated value (using other sources of data) instead of mean

The complete timetable has missing values for the oldest observations. By looking at the data, one notices a certain trend over the years for each variable and each country. This idea is used to fill the missing values. 

Using interpolation to fill the data gaps would be ideal. Since our dataset consists of MultiIndexes, a linear interpolation is the only possible option. However, linear interpolation is not useful in this situation, since the missing values occur only in the oldest observation. Those values are unfortunately not linearly interpolated, simply because there is no value to base the 'inter'polation on.

Hence, 'bfill' is used to fill the data gaps. A possible improvement here would be to create a dataframe using a single index (e.g. a dataframe per country and 'Year' as index) and to perform a spline interpolation, which can extrapolate NaNs at the tail of a Series/Dataframe.

In [23]:
# # Could be done like this if dataframe has no MultiIndexes
# rev_table = table.iloc[::-1]
# rev_table_preprocessed = rev_table.interpolate(method='spline', order=2, axis=0)
# table = rev_table_preprocessed.iloc[::-1]

table_preprocessed = table.fillna(method='bfill')
table_preprocessed.head(21)

Variable Name       Agricultural water withdrawal as % of total renewable water resources  \
Area        Year                                                                            
Afghanistan 1992.0                                          35.498101                       
            1997.0                                          31.427856                       
            2002.0                                          30.613807                       
            2007.0                                          30.613807                       
            2012.0                                          30.613807                       
            2017.0                                          30.613807                       
            2018.0                                          30.613807                       
Argentina   1992.0                                           2.455948                       
            1997.0                                           2.455948                       
            2002.0                                           2.588954                       
            2007.0                                           2.921470                       
            2012.0                                           3.187483                       
            2017.0                                           3.187483                       
            2018.0                                           3.187483                       
Australia   1992.0                                           3.131301                       
            1997.0                                           3.131301                       
            2002.0                                           2.913110                       
            2007.0                                           1.987195                       
            2012.0                                           1.948577                       
            2017.0                                           2.250000                       
            2018.0                                           2.444073                       

Variable Name       Agriculture, value added (% GDP)  Average Temperature  \
Area        Year                                                            
Afghanistan 1992.0                         54.207106                12.61   
            1997.0                         62.416768                12.86   
            2002.0                         45.134344                13.91   
            2007.0                         35.265942                13.80   
            2012.0                         28.556142                13.25   
            2017.0                         25.574670                13.12   
            2018.0                         22.440899                14.55   
Argentina   1992.0                          5.193747                14.37   
            1997.0                          4.527197                15.14   
            2002.0                          8.848660                14.60   
            2007.0                          7.452354                14.20   
            2012.0                          5.781744                15.20   
            2017.0                          5.478382                15.30   
            2018.0                          6.097129                14.99   
Australia   1992.0                          3.263718                21.75   
            1997.0                          3.038954                21.82   
            2002.0                          2.888534                22.25   
            2007.0                          2.338630                22.08   
            2012.0                          2.278906                21.83   
            2017.0                          2.458346                22.57   
            2018.0                          2.121597                22.58   

Variable Name       Environmental Flow Requirements  GDP per capita  \
Area        Year 

In [24]:
# # Could be done like this if dataframe has no MultiIndexes
# rev_table = table.iloc[::-1]
# rev_table_preprocessed = rev_table.interpolate(method='spline', order=2, axis=0)
# table = rev_table_preprocessed.iloc[::-1]

table_preprocessed = table.fillna(method='bfill')
table_preprocessed.head(21)

Variable Name       Agricultural water withdrawal as % of total renewable water resources  \
Area        Year                                                                            
Afghanistan 1992.0                                          35.498101                       
            1997.0                                          31.427856                       
            2002.0                                          30.613807                       
            2007.0                                          30.613807                       
            2012.0                                          30.613807                       
            2017.0                                          30.613807                       
            2018.0                                          30.613807                       
Argentina   1992.0                                           2.455948                       
            1997.0                                           2.455948                       
            2002.0                                           2.588954                       
            2007.0                                           2.921470                       
            2012.0                                           3.187483                       
            2017.0                                           3.187483                       
            2018.0                                           3.187483                       
Australia   1992.0                                           3.131301                       
            1997.0                                           3.131301                       
            2002.0                                           2.913110                       
            2007.0                                           1.987195                       
            2012.0                                           1.948577                       
            2017.0                                           2.250000                       
            2018.0                                           2.444073                       

Variable Name       Agriculture, value added (% GDP)  Average Temperature  \
Area        Year                                                            
Afghanistan 1992.0                         54.207106                12.61   
            1997.0                         62.416768                12.86   
            2002.0                         45.134344                13.91   
            2007.0                         35.265942                13.80   
            2012.0                         28.556142                13.25   
            2017.0                         25.574670                13.12   
            2018.0                         22.440899                14.55   
Argentina   1992.0                          5.193747                14.37   
            1997.0                          4.527197                15.14   
            2002.0                          8.848660                14.60   
            2007.0                          7.452354                14.20   
            2012.0                          5.781744                15.20   
            2017.0                          5.478382                15.30   
            2018.0                          6.097129                14.99   
Australia   1992.0                          3.263718                21.75   
            1997.0                          3.038954                21.82   
            2002.0                          2.888534                22.25   
            2007.0                          2.338630                22.08   
            2012.0                          2.278906                21.83   
            2017.0                          2.458346                22.57   
            2018.0                          2.121597                22.58   

Variable Name       Environmental Flow Requirements  GDP per capita  \
Area        Year 

This dataset is now written to a csv file and then put on AWS for convenience and readibility of the other notebook files

In [25]:
result = table_preprocessed.to_csv('preprocessed data.csv')
result_2018 = table_2018_preprocessed.to_csv('preprocessed data 2018.csv')